In [1]:
#| tags: [parameters]
root = r"inputs"

In [2]:
#| echo: false
import pandas as pd
import os
import numpy as np
import FormComparator as comp

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\langhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\langhe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# WHO verbal autopsy comparisons

In [3]:
in_xlsx1 = os.path.join(root, r"WHOVA2016_v1_5_3_XLS_form_for_ODK.xlsx")

In [4]:
in_xlsx2 = os.path.join(root, r"WHOVA2022_XLS_form_for_ODK.xlsx")

## Generic comparison

In [5]:
comparison = comp.FormComparator(cur_xlsx = in_xlsx1,
                                 ref_xlsx = in_xlsx2,
                                 output_dir = "outputs")

📝 Create Form object from WHOVA2016_v1_5_3_XLS_form_for_ODK.xlsx
	 - ℹ️ survey sheet with 14 columns and 600 rows
	 - ℹ️ choices sheet with 4 columns and 301 rows
	 - ℹ️ settings sheet with 5 columns
	 - ℹ️ no entities sheet found
📝 Create Form object from WHOVA2022_XLS_form_for_ODK.xlsx
	 - ℹ️ survey sheet with 18 columns and 509 rows
	 - ℹ️ choices sheet with 4 columns and 272 rows
	 - ℹ️ settings sheet with 5 columns
	 - ℹ️ no entities sheet found
📝 Compare forms and store results in outputs\comparison_results.xlsx
